In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [ ]:
device = torch.device("cuda")

In [ ]:
def corr2d(X,K):
    batch_size,H,W = X.shape
    k_h,k_w = K.shape
    Y = torch.zeros((batch_size,H-k_h+1,W-k_w+1)).to(device)
    for i in range(Y.shape[1]):
        for j in range(Y.shape[2]):
            Y[:,i,j]=(X[:,i:i+k_h,j:j+k_w]*K).sum()
    return Y

def corr2d_multi_in(X,K):
    res = corr2d(X[:,0:,:,:],K[0,:,:])
    for i in range(1,X.shape[1]):
        res += corr2d(X[:,i,:,:],K[i,:,:])
    return res

def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_multi_in(X,k) for k in K],dim=1)

In [ ]:
class MyConv2D(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size):
        super(MyConv2D,self).__init__()
        #初始化卷积层的2个参数：卷积核、偏差
        if isinstance(kernel_size,int):
            kernel_size = (kernel_size,kernel_size)
        self.weight = nn.Parameter(torch.randn(out_channels,in_channels)+kernel_size)
        self.bias = nn.Parameter(torch.randn(out_channels,1,1))
    
    def forward(self,x):
        #X 输入图片 维度(batch_size,C_in,H,W)
        return corr2d_multi_in_out(x,self.weight)+self.bias 

In [ ]:
class MyConvModule(nn.Module):
    def __init__(self):
        super(MyConvModule,self).__init__()
    #定义三层卷积
        self.conv =nn.Sequential(MyConv2D(in_channels=3,out_channels=32,kernel_size=3),
        nn.BatchNorm2d(32),nn.ReLU(inplace=True))

In [ ]:
# 数据增强 镜像操作/直方图均衡化
# car 779 truck 360 bus 218
import os
import cv2 as cv

bus_path = '/home/hatcher/test/deep-learning/VehicleDataset/bus'
car_path = '/home/hatcher/test/deep-learning/VehicleDataset/car'
truck_path = '/home/hatcher/test/deep-learning/VehicleDataset/truck'
bus_pics = os.listdir(bus_path)
car_pics = os.listdir(car_path)
truck_pics = os.listdir(truck_path)
bus_list, car_list, truck_list = [], [], []

for bus_pic in bus_pics:
    img = cv.imread(bus_path+"/"+bus_pic)
    img = cv.resize(img, (32, 32))
    flip_img = cv.flip(img, 1)
    equ_img = cv.equalizeHist(cv.cvtColor(img,cv.COLOR_BGR2GRAY))
    equ_img=cv.cvtColor(equ_img,cv.COLOR_GRAY2BGR)
    img = img/255
    bus_list.append(img)
    flip_img = flip_img/255
    bus_list.append(flip_img)
    equ_img = equ_img/255
    bus_list.append(equ_img)

for car_pic in car_pics:
    img = cv.imread(car_path+"/"+car_pic)
    img = cv.resize(img, (32, 32))
    img = img/255
    car_list.append(img)

for truck_pic in truck_pics:
    img = cv.imread(truck_path+"/"+truck_pic)
    img = cv.resize(img, (32, 32))
    flip_img = cv.flip(img, 1)
    img = img/255
    truck_list.append(img)
    flip_img = flip_img/255
    truck_list.append(flip_img)

In [ ]:
#此时三种车的数据以FloatTensor的形式分别存放在三个列表中
print("bus_list length:",len(bus_list))
print("car_list length:",len(car_list))
print("truck_list length:",len(truck_list))

In [ ]:
bus_length = len(bus_list)
car_length = len(car_list)
truck_length = len(truck_list)
num_bus_train = int(len(bus_list)*0.8)
num_car_train = int(len(car_list)*0.8)
num_truck_train = int(len(truck_list)*0.8)
train_y,test_y=[],[]
for i in range(num_bus_train):
    #train_y.append([1,0,0,0,0,0,0,0,0,0])
    train_y.append([1,0,0])
for i in range(num_car_train):
    #train_y.append([0,1,0,0,0,0,0,0,0,0])
    train_y.append([0,1,0])
for i in range(num_truck_train):
    #train_y.append([0,0,1,0,0,0,0,0,0,0])
    train_y.append([0,0,1])
for i in range(num_bus_train,bus_length):
    #test_y.append([1,0,0,0,0,0,0,0,0,0])
    test_y.append([1,0,0])
for i in range(num_car_train,car_length):
    #test_y.append([0,1,0,0,0,0,0,0,0,0])
    test_y.append([0,1,0])
for i in range(num_truck_train,truck_length):
    #test_y.append([0,0,1,0,0,0,0,0,0,0])
    test_y.append([0,0,1])
train_x,test_x=[],[]
for i in range(num_bus_train):
    train_x.append(bus_list[i])
for i in range(num_car_train):
    train_x.append(car_list[i])
for i in range(num_truck_train):
    train_x.append(truck_list[i])
for i in range(num_bus_train,bus_length):
    test_x.append(bus_list[i])
for i in range(num_car_train,car_length):
    test_x.append(car_list[i])
for i in range(num_truck_train,truck_length):
    test_x.append(truck_list[i])

train_x = torch.Tensor(train_x).permute(0,3,1,2)
test_x = torch.Tensor(test_x).permute(0,3,1,2)
train_y = torch.Tensor(train_y)
test_y = torch.Tensor(test_y)
print("train_x:",train_x.size())
print("test_x:",test_x.size())
print("train_y:",train_y.size())
print("test_y:",test_y.size())

In [ ]:
def SGD(params,lr):
    for param in params:
        param.data-=lr*param.grad

In [ ]:
def relu(X):
    return torch.max(input=X,other=torch.tensor(0.0))